In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
data = pd.read_csv('ts_hr_feb_2020.csv.zip', parse_dates=['date'])

data['hour'] = data['date'].dt.hour


last_sunday = pd.to_datetime("2020-02-23")
train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [4]:
def prep_seqs(df, l=24, h=24):
    X = []
    Y = []
    for i in range(l,df.shape[0]-h):
        f = df.iloc[i-l:i][['sales_value', 'hour']] #hoje
        y = df.iloc[i:i+h]['sales_value'] # 1 dia depois

        X.append(f.values)
        Y.append(y.values)

    X = np.array(X)
    Y = np.array(Y)

    #X = np.expand_dims(X, -1)
    X = np.swapaxes(X, 0,1)
    
    #Y = np.expand_dims(Y, -1)
    #Y = np.swapaxes(Y, 0,2)
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [5]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error

In [11]:
X, Y = prep_seqs(train)
Xt, Yt = prep_seqs(val)

mean_train = X.mean(1, keepdims=True)
mean_train[:, :, 1] = 0
std_train = X.std(1,  unbiased=False, keepdims=True)
std_train[:, :, 1] = 1

meanY_train = Y.mean(0, keepdims=True)
stdY_train = Y.std(0, unbiased=False, keepdims=True)

X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train


Y -= meanY_train
Y /= stdY_train


X = X.cuda()
Y = Y.cuda()


In [15]:
class LSTMRNN(nn.Module):
    def __init__(self, hidden_size_lstm, embedding_size=2):
        super(LSTMRNN, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size_lstm = hidden_size_lstm 
        
        self.hour_emb = nn.Embedding(24, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size+1, self.hidden_size_lstm)
        self.out = nn.Linear(self.hidden_size_lstm, 24)
        
    def forward(self, input, hidden=None):
        hour_emb = self.hour_emb(input[:,:,-1].long().cuda())
        nums = input[:,:,:-1].cuda()
        inputs = torch.cat([nums, hour_emb], dim=2)
        
        output, _ = self.lstm(inputs)
        output = output[-1, :, :]
        
        output = self.out(output)
        return output
    

In [18]:
%%time

def run_rnn(params):
    hidden_size_lstm, embedding_size, lr = params
    hidden_size_lstm = int(hidden_size_lstm)
    embedding_size = int(embedding_size)
    
    
    total_p = np.zeros((Yt.shape[0], Yt.shape[1], 10))

    for seed in range(10):
        torch.manual_seed(seed)
        np.random.seed(seed)

        lstm = LSTMRNN(hidden_size_lstm, embedding_size).cuda()
        criterion = nn.L1Loss()
        lstm_optimizer = optim.Adam(lstm.parameters(), lr=lr)

        for i in range(100):
            lstm.zero_grad()

            o = lstm(X)
            loss = criterion(o, Y)
            loss.backward()
            lstm_optimizer.step()
            #print(loss.item())

            p = lstm(Xt)
            p = p.detach().cpu()

            p *= stdY_train
            p += meanY_train
            p = p.numpy().squeeze()


        total_p[:, :, seed] = p
        
    e = mean_absolute_error(Yt.numpy(), total_p.mean(axis=-1))

    print("\nhidden_size_lstm = {}, embedding_size = {}, lr = {}".format(hidden_size_lstm, embedding_size, lr))
    print("Ensemble Avg Score = {}".format(e))
    
    return e


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


In [ ]:
from skopt import gp_minimize

space = [(10, 200),
         (1, 20),
         (1e-4, 1e-1, 'log-uniform')]

res = gp_minimize(run_rnn, space,random_state=0, verbose=1, n_calls=50)

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 21.2744
Function value obtained: 326.1539
Current minimum: 324.3901

hidden_size_lstm = 178, embedding_size = 3, lr = 0.0023707980142962724
Ensemble Avg Score = 324.39008277809006